In [35]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

from pipeline2 import H5DataReader
from pipeline2.detection import SimpleSingleChannelSpotDetector
from pipeline2.taskgeneration import BoundingBoxLocationGrouper
from pipeline2.util import filter_dict

%matplotlib notebook
plt.rcParams['figure.figsize'] = [7,7]

def blend(img, cmap, contrast=8, alpha=.8):
    
    norm = plt.Normalize()(img)
    blend = cmap(norm)
    
    # old alpha (everything below mode transparent, everything above alpha)
    #freq, edges = np.histogram(img.ravel(), bins=100)
    #mode = edges[np.argmax(freq)+1]
    #blend[:,:,-1] = (img > mode) * alpha
    
    blend[:,:,-1] = norm**(1/contrast) * alpha
    return blend


def do_settings_diff(path, idx1, idx2, measurement=0, systemSettings=False):
    with h5py.File(path, 'r') as fd:
        r = H5DataReader(fd)
        rd1 = r.get_data(idx1)
        rd2 = r.get_data(idx2)
        
        if rd1 is None or rd2 is None:
            return
        
        meta1 = rd1.globalSettings[measurement] if systemSettings else rd1.measurementSettings[measurement]
        meta2 = rd2.globalSettings[measurement] if systemSettings else rd2.measurementSettings[measurement] 
        
        return diff_dicts(meta1, meta2)

def do_h5_visualization(path):
    with h5py.File(path, 'r') as fd:
        plt.figure()    
        r = H5DataReader(fd)
        #r2 = type('',(object,),{'get_data':lambda self : r.get_data(idx)})()
        #det = SimpleSingleChannelSpotDetector(r2, 2, 1.5, 0, 3, 5, False).withPlotDetections().withVerbose()
        #BoundingBoxLocationGrouper(det, [5e-6, 5e-6, 2e-6]).withVerbose().get_locations()

        cms = [plt.cm.Reds_r, plt.cm.Greens_r, plt.cm.Blues_r]
        cols = ['red', 'green', 'blue']

        for idx in r.idxes:

            rd = r.get_data(idx)
            dta = rd.data


            meta = rd.measurementSettings[0]
            offsOldStg = np.array([filter_dict(
                meta, 'ExpControl/scan/range/offsets/coarse/{}/g_off'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            offsOldG = np.array([filter_dict(
                    meta, 'ExpControl/scan/range/{}/g_off'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            offsOld = offsOldStg + offsOldG + np.array([filter_dict(
                    meta, 'ExpControl/scan/range/{}/off'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            lensOld = np.array([filter_dict(
                    meta, 'ExpControl/scan/range/{}/len'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            pszOld = np.array([filter_dict(
                    meta, 'ExpControl/scan/range/{}/psz'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            #left, right, bottom, top
            left = offsOld[0] - (lensOld[0] + pszOld[0]) /2
            right = offsOld[0] + (lensOld[0] + pszOld[0])/2
            bottom = offsOld[1] + (lensOld[1] + pszOld[1])/2
            top = offsOld[1] - (lensOld[1] + pszOld[1])/2

            # project and show
            img = np.squeeze(np.apply_along_axis(np.max, 1, dta[0][0]))
            plt.imshow(blend(img, cms[len(idx)-1]), extent=(left, right, bottom, top))

            # TODO: keep track of min/max and zoom out
            #plt.xlim(0,20)
            #plt.ylim(0,20)

            ax = plt.gca()
            ax.add_artist(Rectangle((left, top), right-left, bottom-top, fill=False, color=cols[len(idx)-1]))

        plt.gca().patch.set_facecolor('black')
    

In [ ]:
path = '/Volumes/cooperation_data/TobiasRagoczy_StamLab/DavidHoerl/20180226_pipeline2test/7d0fcf9627909e8b1afa42d73edbf0fd.h5'

do_h5_visualization(path)

In [20]:
import collections

def diff_dicts(d1, d2):
    keys_in_1 = set(d1.keys()) - set(d2.keys())
    keys_in_2 = set(d2.keys()) - set(d1.keys())
    keys_both = set(d1.keys()) & set(d2.keys())
    
    added_1 = {}
    added_2 = {}
    diffed_1 = {}
    diffed_2 = {}
    
    for k1 in keys_in_1:
        added_1[k1] = d1[k1]
        
    for k2 in keys_in_2:
        added_2[k2] = d2[k2]
        
    for k in keys_both:
        if (isinstance(d1[k], collections.Mapping) and isinstance(d2[k], collections.Mapping)):
            added_1_r, added_2_r, diffed_1_r, diffed_2_r = diff_dicts(d1[k], d2[k])
            
            if added_1_r:
                added_1[k] = added_1_r
            if added_2_r:
                added_2[k] = added_2_r
            if diffed_1_r:
                diffed_1[k] = diffed_1_r
            if diffed_2_r:
                diffed_2[k] = diffed_2_r
            
        else:
            if (d1[k] != d2[k]):
                diffed_1[k] = d1[k]
                diffed_2[k] = d2[k]
            
    return added_1, added_2, diffed_1, diffed_2
    
d1 = {'a':1, 'b':2}
d2 = {'b':3, 'c':4}

diff_dicts(d1,d2)

({'a': 1}, {'c': 4}, {'b': 2}, {'b': 3})

In [55]:
import json
path = '/Volumes/cooperation_data/TobiasRagoczy_StamLab/DavidHoerl/20180226_pipeline2test/7d0fcf9627909e8b1afa42d73edbf0fd.h5'

added_1, added_2, diffed_1, diffed_2 = do_settings_diff(path, (0,1), (0,0,0), systemSettings=False)
print(json.dumps(diffed_2, indent=1))

{
 "MultiIO": {
  "ao": [
   0.0,
   1.5,
   1.5,
   0.0,
   5.0,
   1.3,
   1.0,
   0.002,
   0.0,
   0.0,
   0.0,
   4.6,
   0.0,
   1.0,
   0.0,
   0.0
  ]
 },
 "ExpControl": {
  "scan": {
   "range": {
    "y": {
     "psz": 1e-08,
     "len": 5e-07,
     "off": -5.884989376e-06
    },
    "z": {
     "psz": 1e-08,
     "res_used": 50,
     "res": 50,
     "len": 5e-07,
     "off": -9.290206978e-08
    },
    "line_accu": 5,
    "x": {
     "psz": 1e-08,
     "len": 5e-07,
     "off": -2.250469567e-05
    },
    "mode": 1296
   }
  },
  "gating": {
   "linesteps": {
    "laser_on": [
     [
      false,
      true,
      true,
      false,
      false,
      false,
      false,
      false
     ],
     [
      false,
      false,
      true,
      false,
      false,
      false,
      false,
      false
     ],
     [
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false
     ],
     [
      false,
      false,
      false,
      fa